# Volume Breakout

In [ ]:
"""
Main script to run hyperparameter optimization and sensitivity analysis
for the Know Sure Thing strategy using the portfolio-based evaluation framework.
"""

import logging
import pandas as pd
import mlflow
from hyperopt import hp
import json
import os
import sys
from datetime import datetime
from typing import Any, Dict, List, Optional, Tuple


sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '..')))

try:
    from src.strategies.breakout.volume_breakout_strat import VolumeBreakoutStrategy
    from src.optimizer.strategy_optimizer import StrategyOptimizer
    from src.optimizer.sensitivity_analyzer import SensitivityAnalyzer
    from src.database.config import DatabaseConfig
except ImportError as e:
    print("Error importing modules. Make sure the script is run from the project root")
    print("or the 'src' directory is in the Python path.")
    print(f"Import Error: {e}")
    sys.exit(1)

In [ ]:
# single stock

# Logging Configuration
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S'
)
logger = logging.getLogger(__name__)

# MLflow Configuration
MLFLOW_TRACKING_URI = "file:./mlruns"  # Store MLflow data locally in ./mlruns
RUN_TIMESTAMP = datetime.now().strftime("%Y%m%d_%H%M%S")

# Data Configuration
TICKER_FILE_PATH = "../data/ticker.xlsx" # Path relative to project root
MAX_TICKERS = 10 # Limit tickers for faster testing, set to None to use all


# Backtest Period
START_DATE = "2014-01-01"
END_DATE = "2023-12-31"

# Optimization Settings
CV_FOLDS = 3
MAX_EVALS = 50  # Number of hyperparameter sets to evaluate
OPTIMIZATION_METRIC = 'harmonic_mean' # Portfolio metric to maximize (minus penalty)
N_JOBS = 1 # Use all available CPU cores for fold evaluation within optimizer

# Sensitivity Analysis Settings
RUN_SENSITIVITY = False # Set to False to skip sensitivity analysis
NUMERIC_PERTURBATION = 0.15 # +/- 15% for sensitivity
SENS_SAMPLES_PER_PARAM = 5
SENS_RANDOM_SAMPLES = 20

# --- Define Search Space for Awesome Oscillator ---

# Note: Hyperopt doesn't easily enforce short_period < long_period directly during sampling.
# The optimizer will evaluate invalid combinations, and they will likely fail or perform poorly.
# Strategy itself raises ValueError if short >= long during initialization.

from src.optimizer.search_space import volume_breakout_strat_search_space

# --- Helper Functions ---

def load_tickers(file_path: str, max_tickers: Optional[int] = None) -> List[str]:
    """Loads and formats ticker symbols from an Excel file."""
    logger.info(f"Loading tickers from: {file_path}")
    try:
        tickers_df = pd.read_excel(file_path)
        # Basic validation
        if not all(col in tickers_df.columns for col in ["Security Name", "Exchange"]):
             raise ValueError("Ticker file missing required columns: 'Security Name', 'Exchange'")

        tickers_df = tickers_df.drop_duplicates(subset=["Security Name"]).reset_index(drop=True)

        def add_ticker_suffix(row):
            name = str(row["Security Name"]).strip()
            exchange = str(row["Exchange"]).strip().upper()
            if exchange == "BSE":
                return f"{name}.BO"
            elif exchange == "NSE":
                return f"{name}.NS"
            else:
                logger.warning(f"Unknown exchange '{exchange}' for {name}. Skipping suffix.")
                return name

        tickers_df["Ticker"] = tickers_df.apply(add_ticker_suffix, axis=1)
        ticker_list = tickers_df["Ticker"].unique().tolist()

        logger.info(f"Loaded {len(ticker_list)} unique tickers.")
        if max_tickers and len(ticker_list) > max_tickers:
            logger.warning(f"Limiting tickers to {max_tickers} for this run.")
            ticker_list = ticker_list[:max_tickers]

        if not ticker_list:
             raise ValueError("No tickers loaded.")

        return ticker_list

    except FileNotFoundError:
        logger.error(f"Ticker file not found at: {file_path}")
        raise
    except Exception as e:
        logger.error(f"Error processing ticker file: {e}")
        raise

# --- Main Execution ---

if __name__ == "__main__":
    logger.info("--- Starting Volume Breakout Optimization Script ---")

    # Setup MLflow
    try:
        mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)
        logger.info(f"MLflow tracking URI set to: {MLFLOW_TRACKING_URI}")
    except Exception as e:
        logger.error(f"Failed to set MLflow tracking URI: {e}")
        sys.exit(1)

        # Create MLflow experiment if it doesn't exist
    try:
        experiment_name = f"Volume_Breakout_{RUN_TIMESTAMP}"
        # Check if experiment exists
        experiment = mlflow.get_experiment_by_name(experiment_name)
        if experiment is None:
            # Create new experiment
            experiment_id = mlflow.create_experiment(experiment_name)
            logger.info(f"Created new MLflow experiment: {experiment_name} with ID: {experiment_id}")
        else:
            experiment_id = experiment.experiment_id
            logger.info(f"Using existing MLflow experiment: {experiment_name} with ID: {experiment_id}")
        
        # Set the experiment for subsequent runs
        mlflow.set_experiment(experiment_name)
    except Exception as e:
        logger.error(f"Failed to create or set MLflow experiment: {e}")
        sys.exit(1)

    # Load Tickers
    try:
        tickers_to_run = load_tickers(TICKER_FILE_PATH, MAX_TICKERS)
    except Exception:
        logger.error("Failed to load tickers. Exiting.")
        sys.exit(1)

    # Database Config
    try:
        db_config = DatabaseConfig.default()
        # Optional: Add a check here to ensure DB connection is valid if possible
        logger.info("Database configuration loaded.")
    except Exception as e:
        logger.error(f"Failed to load database configuration: {e}")
        sys.exit(1)

    # --- Run Optimization ---
    optimizer = None
    best_params = {}
    portfolio_performance_report = pd.DataFrame()
    param_history_report = pd.DataFrame()

    logger.info(f"Initializing StrategyOptimizer for {VolumeBreakoutStrategy.__name__}")
    try:
        optimizer = StrategyOptimizer(
            strategy_class=VolumeBreakoutStrategy,
            db_config=db_config,
            search_space=volume_breakout_strat_search_space,
            tickers=tickers_to_run,
            start_date=START_DATE,
            end_date=END_DATE,
            cv_folds=CV_FOLDS,
            max_evals=MAX_EVALS,
            optimization_metric=OPTIMIZATION_METRIC,
            run_name=f"Volume_Breakout_{RUN_TIMESTAMP}",
            n_jobs=N_JOBS
            # risk_thresholds can be customized here if needed, otherwise defaults are used
        )

        logger.info("Starting hyperparameter optimization...")
        best_params, portfolio_performance_report, param_history_report = optimizer.run_optimization()

        if not best_params:
             logger.error("Optimization did not yield valid results. Best parameters not found.")
        else:
             logger.info("--- Optimization Results ---")
             logger.info(f"Best Parameters found:\n{json.dumps(best_params, indent=2)}")
             logger.info(f"\nBest Portfolio Performance Report:\n{portfolio_performance_report.to_string()}")
             logger.info(f"\nParameter History saved (see MLflow artifacts or CSV file). Head:\n{param_history_report.head().to_string()}")

    except Exception as e:
        logger.error(f"An error occurred during optimization: {e}", exc_info=True)
        # Attempt to end MLflow run if it was started by the optimizer
        if mlflow.active_run():
            mlflow.end_run("FAILED")

    # --- Run Sensitivity Analysis (Optional) ---
    if RUN_SENSITIVITY and optimizer and best_params:
        logger.info("\n--- Starting Sensitivity Analysis ---")
        try:
            analyzer = SensitivityAnalyzer(
                strategy_optimizer=optimizer, # Reuse optimizer for its config and evaluation cache
                base_params=best_params,
                numeric_perturbation=NUMERIC_PERTURBATION,
                num_samples_per_param=SENS_SAMPLES_PER_PARAM,
                num_random_samples=SENS_RANDOM_SAMPLES,
                parallel=True # Relies on optimizer's internal parallelization/caching
            )

            sensitivity_results_df, parameter_impact_df = analyzer.run()

            if sensitivity_results_df.empty:
                 logger.warning("Sensitivity analysis did not produce results.")
            else:
                logger.info("--- Sensitivity Analysis Results ---")
                logger.info(f"Sensitivity Results saved (see MLflow artifacts or CSV file). Head:\n{sensitivity_results_df.head().to_string()}")
                logger.info(f"\nParameter Impact Report (Correlation):\n{parameter_impact_df.to_string()}")

        except Exception as e:
            logger.error(f"An error occurred during sensitivity analysis: {e}", exc_info=True)
            if mlflow.active_run():
                 mlflow.end_run("FAILED") # End sensitivity run if it crashed

    elif RUN_SENSITIVITY and (not optimizer or not best_params):
        logger.warning("Skipping sensitivity analysis because optimization failed or produced no best parameters.")


    # Ensure any lingering run is terminated cleanly
    # Should not be necessary if 'with mlflow.start_run()' is used correctly inside modules
    # try:
    #     while mlflow.active_run():
    #         logger.info(f"Ending lingering MLflow run: {mlflow.active_run().info.run_id}")
    #         mlflow.end_run()
    # except Exception:
    #      pass # Ignore errors during cleanup

    logger.info("--- Script Finished ---")

# Triangle Breakout Strategy

In [ ]:
"""
Main script to run hyperparameter optimization and sensitivity analysis
for the Know Sure Thing strategy using the portfolio-based evaluation framework.
"""

import logging
import pandas as pd
import mlflow
from hyperopt import hp
import json
import os
import sys
from datetime import datetime
from typing import Any, Dict, List, Optional, Tuple


sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '..')))

try:
    from src.strategies.breakout.triangle_breakout_strat import TriangleBreakout
    from src.optimizer.strategy_optimizer import StrategyOptimizer
    from src.optimizer.sensitivity_analyzer import SensitivityAnalyzer
    from src.database.config import DatabaseConfig
except ImportError as e:
    print("Error importing modules. Make sure the script is run from the project root")
    print("or the 'src' directory is in the Python path.")
    print(f"Import Error: {e}")
    sys.exit(1)

In [ ]:
# single stock

# Logging Configuration
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S'
)
logger = logging.getLogger(__name__)

# MLflow Configuration
MLFLOW_TRACKING_URI = "file:./mlruns"  # Store MLflow data locally in ./mlruns
RUN_TIMESTAMP = datetime.now().strftime("%Y%m%d_%H%M%S")

# Data Configuration
TICKER_FILE_PATH = "../data/ticker.xlsx" # Path relative to project root
MAX_TICKERS = 10 # Limit tickers for faster testing, set to None to use all


# Backtest Period
START_DATE = "2014-01-01"
END_DATE = "2023-12-31"

# Optimization Settings
CV_FOLDS = 3
MAX_EVALS = 50  # Number of hyperparameter sets to evaluate
OPTIMIZATION_METRIC = 'harmonic_mean' # Portfolio metric to maximize (minus penalty)
N_JOBS = 1 # Use all available CPU cores for fold evaluation within optimizer

# Sensitivity Analysis Settings
RUN_SENSITIVITY = False # Set to False to skip sensitivity analysis
NUMERIC_PERTURBATION = 0.15 # +/- 15% for sensitivity
SENS_SAMPLES_PER_PARAM = 5
SENS_RANDOM_SAMPLES = 20

# --- Define Search Space for Awesome Oscillator ---

# Note: Hyperopt doesn't easily enforce short_period < long_period directly during sampling.
# The optimizer will evaluate invalid combinations, and they will likely fail or perform poorly.
# Strategy itself raises ValueError if short >= long during initialization.

from src.optimizer.search_space import triangle_breakout_strat_search_space

# --- Helper Functions ---

def load_tickers(file_path: str, max_tickers: Optional[int] = None) -> List[str]:
    """Loads and formats ticker symbols from an Excel file."""
    logger.info(f"Loading tickers from: {file_path}")
    try:
        tickers_df = pd.read_excel(file_path)
        # Basic validation
        if not all(col in tickers_df.columns for col in ["Security Name", "Exchange"]):
             raise ValueError("Ticker file missing required columns: 'Security Name', 'Exchange'")

        tickers_df = tickers_df.drop_duplicates(subset=["Security Name"]).reset_index(drop=True)

        def add_ticker_suffix(row):
            name = str(row["Security Name"]).strip()
            exchange = str(row["Exchange"]).strip().upper()
            if exchange == "BSE":
                return f"{name}.BO"
            elif exchange == "NSE":
                return f"{name}.NS"
            else:
                logger.warning(f"Unknown exchange '{exchange}' for {name}. Skipping suffix.")
                return name

        tickers_df["Ticker"] = tickers_df.apply(add_ticker_suffix, axis=1)
        ticker_list = tickers_df["Ticker"].unique().tolist()

        logger.info(f"Loaded {len(ticker_list)} unique tickers.")
        if max_tickers and len(ticker_list) > max_tickers:
            logger.warning(f"Limiting tickers to {max_tickers} for this run.")
            ticker_list = ticker_list[:max_tickers]

        if not ticker_list:
             raise ValueError("No tickers loaded.")

        return ticker_list

    except FileNotFoundError:
        logger.error(f"Ticker file not found at: {file_path}")
        raise
    except Exception as e:
        logger.error(f"Error processing ticker file: {e}")
        raise

# --- Main Execution ---

if __name__ == "__main__":
    logger.info("--- Starting Triangle Breakout Optimization Script ---")

    # Setup MLflow
    try:
        mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)
        logger.info(f"MLflow tracking URI set to: {MLFLOW_TRACKING_URI}")
    except Exception as e:
        logger.error(f"Failed to set MLflow tracking URI: {e}")
        sys.exit(1)

        # Create MLflow experiment if it doesn't exist
    try:
        experiment_name = f"Volume_Breakout_{RUN_TIMESTAMP}"
        # Check if experiment exists
        experiment = mlflow.get_experiment_by_name(experiment_name)
        if experiment is None:
            # Create new experiment
            experiment_id = mlflow.create_experiment(experiment_name)
            logger.info(f"Created new MLflow experiment: {experiment_name} with ID: {experiment_id}")
        else:
            experiment_id = experiment.experiment_id
            logger.info(f"Using existing MLflow experiment: {experiment_name} with ID: {experiment_id}")
        
        # Set the experiment for subsequent runs
        mlflow.set_experiment(experiment_name)
    except Exception as e:
        logger.error(f"Failed to create or set MLflow experiment: {e}")
        sys.exit(1)

    # Load Tickers
    try:
        tickers_to_run = load_tickers(TICKER_FILE_PATH, MAX_TICKERS)
    except Exception:
        logger.error("Failed to load tickers. Exiting.")
        sys.exit(1)

    # Database Config
    try:
        db_config = DatabaseConfig.default()
        # Optional: Add a check here to ensure DB connection is valid if possible
        logger.info("Database configuration loaded.")
    except Exception as e:
        logger.error(f"Failed to load database configuration: {e}")
        sys.exit(1)

    # --- Run Optimization ---
    optimizer = None
    best_params = {}
    portfolio_performance_report = pd.DataFrame()
    param_history_report = pd.DataFrame()

    logger.info(f"Initializing StrategyOptimizer for {TriangleBreakout.__name__}")
    try:
        optimizer = StrategyOptimizer(
            strategy_class=TriangleBreakout,
            db_config=db_config,
            search_space=triangle_breakout_strat_search_space,
            tickers=tickers_to_run,
            start_date=START_DATE,
            end_date=END_DATE,
            cv_folds=CV_FOLDS,
            max_evals=MAX_EVALS,
            optimization_metric=OPTIMIZATION_METRIC,
            run_name=f"Volume_Breakout_{RUN_TIMESTAMP}",
            n_jobs=N_JOBS
            # risk_thresholds can be customized here if needed, otherwise defaults are used
        )

        logger.info("Starting hyperparameter optimization...")
        best_params, portfolio_performance_report, param_history_report = optimizer.run_optimization()

        if not best_params:
             logger.error("Optimization did not yield valid results. Best parameters not found.")
        else:
             logger.info("--- Optimization Results ---")
             logger.info(f"Best Parameters found:\n{json.dumps(best_params, indent=2)}")
             logger.info(f"\nBest Portfolio Performance Report:\n{portfolio_performance_report.to_string()}")
             logger.info(f"\nParameter History saved (see MLflow artifacts or CSV file). Head:\n{param_history_report.head().to_string()}")

    except Exception as e:
        logger.error(f"An error occurred during optimization: {e}", exc_info=True)
        # Attempt to end MLflow run if it was started by the optimizer
        if mlflow.active_run():
            mlflow.end_run("FAILED")

    # --- Run Sensitivity Analysis (Optional) ---
    if RUN_SENSITIVITY and optimizer and best_params:
        logger.info("\n--- Starting Sensitivity Analysis ---")
        try:
            analyzer = SensitivityAnalyzer(
                strategy_optimizer=optimizer, # Reuse optimizer for its config and evaluation cache
                base_params=best_params,
                numeric_perturbation=NUMERIC_PERTURBATION,
                num_samples_per_param=SENS_SAMPLES_PER_PARAM,
                num_random_samples=SENS_RANDOM_SAMPLES,
                parallel=True # Relies on optimizer's internal parallelization/caching
            )

            sensitivity_results_df, parameter_impact_df = analyzer.run()

            if sensitivity_results_df.empty:
                 logger.warning("Sensitivity analysis did not produce results.")
            else:
                logger.info("--- Sensitivity Analysis Results ---")
                logger.info(f"Sensitivity Results saved (see MLflow artifacts or CSV file). Head:\n{sensitivity_results_df.head().to_string()}")
                logger.info(f"\nParameter Impact Report (Correlation):\n{parameter_impact_df.to_string()}")

        except Exception as e:
            logger.error(f"An error occurred during sensitivity analysis: {e}", exc_info=True)
            if mlflow.active_run():
                 mlflow.end_run("FAILED") # End sensitivity run if it crashed

    elif RUN_SENSITIVITY and (not optimizer or not best_params):
        logger.warning("Skipping sensitivity analysis because optimization failed or produced no best parameters.")


    # Ensure any lingering run is terminated cleanly
    # Should not be necessary if 'with mlflow.start_run()' is used correctly inside modules
    # try:
    #     while mlflow.active_run():
    #         logger.info(f"Ending lingering MLflow run: {mlflow.active_run().info.run_id}")
    #         mlflow.end_run()
    # except Exception:
    #      pass # Ignore errors during cleanup

    logger.info("--- Script Finished ---")

# Cup and Handle Strategy

In [ ]:
"""
Main script to run hyperparameter optimization and sensitivity analysis
for the Know Sure Thing strategy using the portfolio-based evaluation framework.
"""

import logging
import pandas as pd
import mlflow
from hyperopt import hp
import json
import os
import sys
from datetime import datetime
from typing import Any, Dict, List, Optional, Tuple


sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '..')))

try:
    from src.strategies.breakout.cup_and_handle_strat import CupAndHandle
    from src.optimizer.strategy_optimizer import StrategyOptimizer
    from src.optimizer.sensitivity_analyzer import SensitivityAnalyzer
    from src.database.config import DatabaseConfig
except ImportError as e:
    print("Error importing modules. Make sure the script is run from the project root")
    print("or the 'src' directory is in the Python path.")
    print(f"Import Error: {e}")
    sys.exit(1)

In [ ]:
# single stock

# Logging Configuration
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S'
)
logger = logging.getLogger(__name__)

# MLflow Configuration
MLFLOW_TRACKING_URI = "file:./mlruns"  # Store MLflow data locally in ./mlruns
RUN_TIMESTAMP = datetime.now().strftime("%Y%m%d_%H%M%S")

# Data Configuration
TICKER_FILE_PATH = "../data/ticker.xlsx" # Path relative to project root
MAX_TICKERS = 10 # Limit tickers for faster testing, set to None to use all


# Backtest Period
START_DATE = "2014-01-01"
END_DATE = "2023-12-31"

# Optimization Settings
CV_FOLDS = 3
MAX_EVALS = 50  # Number of hyperparameter sets to evaluate
OPTIMIZATION_METRIC = 'harmonic_mean' # Portfolio metric to maximize (minus penalty)
N_JOBS = 1 # Use all available CPU cores for fold evaluation within optimizer

# Sensitivity Analysis Settings
RUN_SENSITIVITY = False # Set to False to skip sensitivity analysis
NUMERIC_PERTURBATION = 0.15 # +/- 15% for sensitivity
SENS_SAMPLES_PER_PARAM = 5
SENS_RANDOM_SAMPLES = 20

# --- Define Search Space for Awesome Oscillator ---

# Note: Hyperopt doesn't easily enforce short_period < long_period directly during sampling.
# The optimizer will evaluate invalid combinations, and they will likely fail or perform poorly.
# Strategy itself raises ValueError if short >= long during initialization.

from src.optimizer.search_space import cup_and_handle_strat_search_space

# --- Helper Functions ---

def load_tickers(file_path: str, max_tickers: Optional[int] = None) -> List[str]:
    """Loads and formats ticker symbols from an Excel file."""
    logger.info(f"Loading tickers from: {file_path}")
    try:
        tickers_df = pd.read_excel(file_path)
        # Basic validation
        if not all(col in tickers_df.columns for col in ["Security Name", "Exchange"]):
             raise ValueError("Ticker file missing required columns: 'Security Name', 'Exchange'")

        tickers_df = tickers_df.drop_duplicates(subset=["Security Name"]).reset_index(drop=True)

        def add_ticker_suffix(row):
            name = str(row["Security Name"]).strip()
            exchange = str(row["Exchange"]).strip().upper()
            if exchange == "BSE":
                return f"{name}.BO"
            elif exchange == "NSE":
                return f"{name}.NS"
            else:
                logger.warning(f"Unknown exchange '{exchange}' for {name}. Skipping suffix.")
                return name

        tickers_df["Ticker"] = tickers_df.apply(add_ticker_suffix, axis=1)
        ticker_list = tickers_df["Ticker"].unique().tolist()

        logger.info(f"Loaded {len(ticker_list)} unique tickers.")
        if max_tickers and len(ticker_list) > max_tickers:
            logger.warning(f"Limiting tickers to {max_tickers} for this run.")
            ticker_list = ticker_list[:max_tickers]

        if not ticker_list:
             raise ValueError("No tickers loaded.")

        return ticker_list

    except FileNotFoundError:
        logger.error(f"Ticker file not found at: {file_path}")
        raise
    except Exception as e:
        logger.error(f"Error processing ticker file: {e}")
        raise

# --- Main Execution ---

if __name__ == "__main__":
    logger.info("--- Starting Cup and Handle Optimization Script ---")

    # Setup MLflow
    try:
        mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)
        logger.info(f"MLflow tracking URI set to: {MLFLOW_TRACKING_URI}")
    except Exception as e:
        logger.error(f"Failed to set MLflow tracking URI: {e}")
        sys.exit(1)

        # Create MLflow experiment if it doesn't exist
    try:
        experiment_name = f"{CupAndHandle.__name__}_{RUN_TIMESTAMP}"
        # Check if experiment exists
        experiment = mlflow.get_experiment_by_name(experiment_name)
        if experiment is None:
            # Create new experiment
            experiment_id = mlflow.create_experiment(experiment_name)
            logger.info(f"Created new MLflow experiment: {experiment_name} with ID: {experiment_id}")
        else:
            experiment_id = experiment.experiment_id
            logger.info(f"Using existing MLflow experiment: {experiment_name} with ID: {experiment_id}")
        
        # Set the experiment for subsequent runs
        mlflow.set_experiment(experiment_name)
    except Exception as e:
        logger.error(f"Failed to create or set MLflow experiment: {e}")
        sys.exit(1)

    # Load Tickers
    try:
        tickers_to_run = load_tickers(TICKER_FILE_PATH, MAX_TICKERS)
    except Exception:
        logger.error("Failed to load tickers. Exiting.")
        sys.exit(1)

    # Database Config
    try:
        db_config = DatabaseConfig.default()
        # Optional: Add a check here to ensure DB connection is valid if possible
        logger.info("Database configuration loaded.")
    except Exception as e:
        logger.error(f"Failed to load database configuration: {e}")
        sys.exit(1)

    # --- Run Optimization ---
    optimizer = None
    best_params = {}
    portfolio_performance_report = pd.DataFrame()
    param_history_report = pd.DataFrame()

    logger.info(f"Initializing StrategyOptimizer for {CupAndHandle.__name__}")
    try:
        optimizer = StrategyOptimizer(
            strategy_class=CupAndHandle,
            db_config=db_config,
            search_space=cup_and_handle_strat_search_space,
            tickers=tickers_to_run,
            start_date=START_DATE,
            end_date=END_DATE,
            cv_folds=CV_FOLDS,
            max_evals=MAX_EVALS,
            optimization_metric=OPTIMIZATION_METRIC,
            run_name=experiment_name,
            n_jobs=N_JOBS
            # risk_thresholds can be customized here if needed, otherwise defaults are used
        )

        logger.info("Starting hyperparameter optimization...")
        best_params, portfolio_performance_report, param_history_report = optimizer.run_optimization()

        if not best_params:
             logger.error("Optimization did not yield valid results. Best parameters not found.")
        else:
             logger.info("--- Optimization Results ---")
             logger.info(f"Best Parameters found:\n{json.dumps(best_params, indent=2)}")
             logger.info(f"\nBest Portfolio Performance Report:\n{portfolio_performance_report.to_string()}")
             logger.info(f"\nParameter History saved (see MLflow artifacts or CSV file). Head:\n{param_history_report.head().to_string()}")

    except Exception as e:
        logger.error(f"An error occurred during optimization: {e}", exc_info=True)
        # Attempt to end MLflow run if it was started by the optimizer
        if mlflow.active_run():
            mlflow.end_run("FAILED")

    # --- Run Sensitivity Analysis (Optional) ---
    if RUN_SENSITIVITY and optimizer and best_params:
        logger.info("\n--- Starting Sensitivity Analysis ---")
        try:
            analyzer = SensitivityAnalyzer(
                strategy_optimizer=optimizer, # Reuse optimizer for its config and evaluation cache
                base_params=best_params,
                numeric_perturbation=NUMERIC_PERTURBATION,
                num_samples_per_param=SENS_SAMPLES_PER_PARAM,
                num_random_samples=SENS_RANDOM_SAMPLES,
                parallel=True # Relies on optimizer's internal parallelization/caching
            )

            sensitivity_results_df, parameter_impact_df = analyzer.run()

            if sensitivity_results_df.empty:
                 logger.warning("Sensitivity analysis did not produce results.")
            else:
                logger.info("--- Sensitivity Analysis Results ---")
                logger.info(f"Sensitivity Results saved (see MLflow artifacts or CSV file). Head:\n{sensitivity_results_df.head().to_string()}")
                logger.info(f"\nParameter Impact Report (Correlation):\n{parameter_impact_df.to_string()}")

        except Exception as e:
            logger.error(f"An error occurred during sensitivity analysis: {e}", exc_info=True)
            if mlflow.active_run():
                 mlflow.end_run("FAILED") # End sensitivity run if it crashed

    elif RUN_SENSITIVITY and (not optimizer or not best_params):
        logger.warning("Skipping sensitivity analysis because optimization failed or produced no best parameters.")


    # Ensure any lingering run is terminated cleanly
    # Should not be necessary if 'with mlflow.start_run()' is used correctly inside modules
    # try:
    #     while mlflow.active_run():
    #         logger.info(f"Ending lingering MLflow run: {mlflow.active_run().info.run_id}")
    #         mlflow.end_run()
    # except Exception:
    #      pass # Ignore errors during cleanup

    logger.info("--- Script Finished ---")